In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from JSONreader import *
import pandas as pd
import os
import progressbar
from glob import glob
import json

In [103]:
MAIN_FOLDER = 'F:/DATASETS/'

def get_subfolders(path):
    return [x[0] for x in os.walk(path)][1:]

def collect_some_json(path, num):
    subfolder_list = get_subfolders(MAIN_FOLDER + "/" + path)
    print(subfolder_list)
    df = pd.DataFrame()
    for sub in subfolder_list:
        temp_df = json_database_to_dataframe(sub, int(round(num/len(subfolder_list))), path)
        df = pd.concat([df, temp_df])

    return df

folders = next(os.walk(MAIN_FOLDER))[1]
folders

['English news articles',
 'Entertainment',
 'Financial',
 'Political',
 'Sports',
 'Technology',
 'Travel',
 'World']

In [113]:
df = pd.DataFrame()
for f in folders[5:6]:
    print(f)
    temp_df = collect_some_json(f,1000)
    df = pd.concat([df, temp_df])

  1% (7 of 500) |                         | Elapsed Time: 0:00:00 ETA:  0:00:00

Technology
['F:/DATASETS//Technology\\663_webhose-2015-09-new_20170904095535', 'F:/DATASETS//Technology\\663_webhose-2015-10-new_20170904095703']


100% (500 of 500) |#######################| Elapsed Time: 0:00:03 Time: 0:00:03
100% (500 of 500) |#######################| Elapsed Time: 0:00:03 Time: 0:00:03


In [114]:
df_domain_categories = pd.DataFrame(columns  =['site', 'similarsite_category', 'similarsite_subcategory'])
with progressbar.ProgressBar(max_value=len(df.site_full.unique())) as bar:       
    for i, domain in enumerate(df.site_full.unique()):
        temp_cat = get_category(domain)
        df_domain_categories.loc[i] = [domain, temp_cat[0], temp_cat[1]]
        bar.update(i)

 10% (19 of 187) |##                      | Elapsed Time: 0:00:17 ETA:  0:02:24

no soup


 20% (38 of 187) |####                    | Elapsed Time: 0:00:33 ETA:  0:02:10

no soup


 25% (47 of 187) |######                  | Elapsed Time: 0:00:41 ETA:  0:01:56

...

 34% (65 of 187) |########                | Elapsed Time: 0:00:56 ETA:  0:01:43

no soup


 88% (166 of 187) |####################   | Elapsed Time: 0:02:30 ETA:  0:00:20

no soup


100% (187 of 187) |#######################| Elapsed Time: 0:02:47 Time: 0:02:47


In [116]:
df_w_categories = pd.merge(df, df_domain_categories, left_on='site_full', right_on='site')
t = df_w_categories.groupby(df_w_categories.similarsite_subcategory).count()
t.loc[t['similarsite_category'] >= 10]

,site_full,site_section,section_title,url,country,domain_rank,title,performance_score,site_x,participants_count,...,published,replies_count,author,highlightText,language,text,webhose_category,site_y,similarsite_category,similarsite_subcategory
similarsite_subcategory,,,,,,,,,,,,,,,,,,,,,
,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
Business News,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
Government,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
Investing,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
Magazines And E-Zines,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Marketing And Advertising,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Newspapers,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
Online Marketing,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
TV And Video,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45


In [111]:
from bs4 import BeautifulSoup
import requests
import sys

SIMILARSITES = 'https://www.similarsites.com/site/'

def get_category(domain):
    
    target_url = SIMILARSITES + domain

    result = requests.get(target_url)
    
    if(result.status_code == 200):
        c = result.content
        soup = BeautifulSoup(c,"html.parser")
    else:
        print("no soup")
        return ['',''] 
            
    container = soup.find("div", { "class" : "info-details" })
    category_subcategory = container.findAll("span", {"class": "badge thin"})
    
    category=''
    subcategory=''
    
    num = len(category_subcategory)
    
    if num >= 1:
        category = category_subcategory[0].text.lstrip().rstrip()
        
    if num >=2:
        subcategory = category_subcategory[1].text.lstrip().rstrip()
    
    return pd.Series([category, subcategory])

get_category('www.latimes.com')

0    News And Media
1        Newspapers
dtype: object

In [61]:
from newspaper import Article

b = Article(df.loc[5].url)
b.download()
b.parse()
b.nlp()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [28]:
def get_article_features(art):
    art_dict = art.meta_data
    
    description = art_dict['description']
    keywords = art_dict['keywords']
    #num_images = len(b.meta_data['og']['image'])

In [26]:
(b.meta_data['og']['image'])

{'height': 630,
 'identifier': 'https://cbsnews1.cbsistatic.com/hub/i/r/2016/11/19/750683b0-b0d5-4729-9bc4-8bc7dbb1deca/thumbnail/1200x630/09fb78ab61246586fa339ef144a77b08/1119-fbm-fbn-newcabinet-1183580-640x360.jpg',
 'width': 1200}

In [29]:
b.meta_data

defaultdict(dict,
            {'al': {'ipad': {'app_name': 'ABC News for iPad',
               'app_store_id': 380520716,
               'url': 'abcnewsipad://link/home'},
              'iphone': {'app_name': 'ABC News - Breaking US & World News',
               'app_store_id': 300255638,
               'url': 'abcnewsiphone://link/home'}},
             'application-name': 'ABC News',
             'author': 'ABC News',
             'copyright': 'Copyright (c) 2017 ABC News Internet Ventures',
             'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com',
             'fb': {'admins': 704409894,
              'app_id': 4942312939,
              'pages': 55845270277},
             'fb_title': 'ABC News',
             'google-site-verification': 'jdEUqeNIqEwginpWLQdAonWQhUVB5jq9pQ03R3YsFL4',
             'keywords': 'ABC News, breaking news, headline news, latest news, top news, online news, video news, world news